In [1]:
!pip install awswrangler
!pip install optuna

import awswrangler as wr

import mlflow

import datetime
import optuna

from mlflow.models import infer_signature
from mlflow_aux import get_or_create_experiment

from optuna_aux import champion_callback, objective

from sklearn.svm import SVC 
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split


from plots import plot_correlation_with_target, plot_information_gain_with_target
import pandas as pd
import numpy as np
import awswrangler as wr

import datetime
#import optuna

from mlflow.models import infer_signature
#from mlflow_aux import get_or_create_experiment

#from optuna_aux import champion_callback, objective



# Optuna es un poco verboso, dejamos que solo nos muestre logs de errores
#optuna.logging.set_verbosity(optuna.logging.ERROR)


# Optuna es un poco verboso, dejamos que solo nos muestre logs de errores
optuna.logging.set_verbosity(optuna.logging.ERROR)

# Para que funciones, todos nuestros scripts debemos exportar las siguientes variables de entorno
%env AWS_ACCESS_KEY_ID=minio   
%env AWS_SECRET_ACCESS_KEY=minio123 
%env MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
%env AWS_ENDPOINT_URL_S3=http://localhost:9000


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip
c:\python38\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


env: AWS_ACCESS_KEY_ID=minio
env: AWS_SECRET_ACCESS_KEY=minio123
env: MLFLOW_S3_ENDPOINT_URL=http://localhost:9000
env: AWS_ENDPOINT_URL_S3=http://localhost:9000


In [2]:
mlflow_server = "http://localhost:5000"
#mlflow_server = "http://192.168.0.21:5000"

mlflow.set_tracking_uri(mlflow_server)

In [3]:


data_path = "s3://data/raw/mnist_784.csv"
#dataframe = mnist_784_dataset.data.original

#target_col = Variable.get("target_col_mnist_784")

mnist = fetch_openml('mnist_784', as_frame = False)

dataframe = pd.DataFrame(mnist.data, columns=mnist.feature_names)
target_col = pd.DataFrame(mnist.target, columns=['label'])

dataframe['label'] = target_col['label']

wr.s3.to_csv(df=dataframe,
            path="s3://data/raw/mnist_784.csv",
            index=False)

X_digits=dataframe.drop(['label'], axis=1)
y_digits=dataframe['label']

X_train, X_test, y_train, y_test = train_test_split(X_digits,
                                                y_digits,
                                                test_size=0.3,
                                                random_state=32)

# Dado que estamos usando como tracking a MLFlow, mostrar los gráficos aquí no tiene sentido.
#correlation_plot = plot_correlation_with_target(X_train, y_train)
#information_gain_plot = plot_information_gain_with_target(X_train, y_train)


wr.s3.to_csv(df=X_train,
            path="s3://data/final/train/mnist_784_X_train.csv",
            index=False)

wr.s3.to_csv(df=X_test,
            path="s3://data/final/test/mnist_784_X_test.csv",
            index=False)

wr.s3.to_csv(df=y_train,
            path="s3://data/final/train/mnist_784_y_train.csv",
            index=False)

wr.s3.to_csv(df=y_test,
            path="s3://data/final/test/mnist_784_y_test.csv",
            index=False)

c:\python38\lib\site-packages\sklearn\datasets\_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


{'paths': ['s3://data/final/test/mnist_784_y_test.csv'],
 'partitions_values': {}}

# Búsqueda de mejor modelo e hiperparámetros

Dado nuestro dataset de mnist_784, el cual ya pasó por el proceso de ETL y se encuentra en nuestro S3 bucket, vamos a realizar una búsqueda de cual seria el mejor modelo y que hiperparametros usar.

La búsqueda de hiperparametros la haremos usando Optuna y el tracking será realizado mediante MLFlow.

OBS: Para la confección de esta notebook, nos basamos en el tutorial de [MLFlow](https://mlflow.org/docs/latest/traditional-ml/hyperparameter-tuning-with-child-runs/notebooks/index.html).

In [4]:
# Cargamos los datos para realizar nuestro estudio.
# OBS, no vamos a cargar los datos de testing, nada de Data leakage por aquí
X_train =  wr.s3.read_csv("s3://data/final/train/mnist_784_X_train.csv").to_numpy()
y_train =  wr.s3.read_csv("s3://data/final/train/mnist_784_y_train.csv").to_numpy()

X_test =  wr.s3.read_csv("s3://data/final/test/mnist_784_X_test.csv").to_numpy()
y_test =  wr.s3.read_csv("s3://data/final/test/mnist_784_y_test.csv").to_numpy()

## Investigamos la correlación de features con la variable objetivo

Antes de profundizar en el proceso de construcción de modelo, es esencial comprender las relaciones entre nuestras features  y la variable objetivo. Por lo que vamos a realizar un gráfico que indica el coeficiente de correlación de cada feature en relación con la variable objetivo. Esto nos sirve para:

- Evitar data leakage: Debemos asegurarnos de que ninguna característica se correlacione perfectamente con el objetivo (un coeficiente de correlación de aproximadamente 1.0). Si existe tal correlación, es una señal de que nuestro conjunto de datos podría estar "filtrando" información sobre el objetivo. 

- Garantizar relaciones significativas: Idealmente, nuestras características deberían tener algún grado de correlación con el objetivo. Inclusive si estamos trabajando con un problema de clasificación, aunque los resultados no son tan importantes como en un caso de regresión.

- Auditoría y trazabilidad: Loggear esta visualización de correlación con nuestra ejecución principal de MLflow garantiza la trazabilidad. Proporciona una instantánea de las características de los datos en el momento del entrenamiento del modelo, lo cual es invaluable para propósitos de auditoría y replicabilidad.

In [5]:
#from plots import plot_correlation_with_target, plot_information_gain_with_target

In [6]:
# Dado que estamos usando como tracking a MLFlow, mostrar los gráficos aquí no tiene sentido.
#correlation_plot = plot_correlation_with_target(X_train, y_train)
#information_gain_plot = plot_information_gain_with_target(X_train, y_train)

## Arrancamos a experimentar

Antes de poder realizar experimentos, vamos a crear el experimento en MLFLow, pero para evitar desorden, vamos a usar una función que se fije primero si el experimento existe, si esto es así, devuelve su ID.

Además creamos el nombre del run padre con el que vamos a ir registrando las ejecuciones.

In [7]:
# Creemos el experimento
experiment_id = get_or_create_experiment("mnist_784")
print(experiment_id)

run_name_parent = "best_hyperparam_"  + datetime.datetime.today().strftime('%Y/%m/%d-%H:%M:%S"')

1


Ya con todo seteado, vamos a ejecutar la optimización usando Optuna, el cual realiza una búsqueda Bayesiana, la cual es más eficiente que una búsqueda de grilla tradicional. La desventaja es que es más difícil de paralelizar.

In [8]:
with mlflow.start_run(experiment_id=experiment_id, run_name=run_name_parent, nested=True):
    # Inicializamos el estudio de Optuna

    #artifact_path = "s3://data/model"
    artifact_path = "/model"

    #mlflow.log_artifact("features.txt", artifact_path=artifact_path)

    study = optuna.create_study(direction="maximize")

    # Ejecutamos los trials de optimización de hiperparametros. Cada uno de estos trials se ejecuta con un run separado, pero 
    # está anidado al run padre.
    # Notar la adición del `champion_callback` para controlar qué mensajes mostramos
    # Para entender mejor esto ver la documentación de objective y champion_callback en optuna_aux
    #study.optimize(lambda trial: objective(trial, X_train, y_train, experiment_id), n_trials=250, callbacks=[champion_callback])

    # Una vez que terminamos la búsqueda, guardamos los mejores parámetros en el run padre.
    #mlflow.log_params(study.best_params)
    mlflow.log_params({'tree_max_depth':10})
    #mlflow.log_metric("best_train_f1", study.best_value)

    mlflow.set_tags(
        tags={
            "project": "mnist_784",
            #"optimizer_engine": "optuna",
            "model_family": "sklearn",
            "feature_set_version": 1,
        }
    )

    # Una vez que terminamos la búsqueda, nos quedamos con el mejor modelo y lo entrenamos

    model = DecisionTreeClassifier(criterion = 'entropy', max_depth=10)
    #model = DecisionTreeClassifier(criterion = 'entropy', max_depth=study.best_params["tree_max_depth"])


    model = model.fit(X_train, y_train)

    # Y testeamos el modelo y logueamos el resultado
    y_pred = model.predict(X_test)
    #f1_score = f1_score(y_test.to_numpy().ravel(), y_pred, average='weighted')
    accuracy_score = accuracy_score(y_test, y_pred)
    
    #mlflow.log_metric("test_f1", f1_score)
    mlflow.log_metric("test_accuracy_score", accuracy_score)

    # Logueamos los artefactos de las gráficas de correlación y de information_gain
    #mlflow.log_figure(figure=correlation_plot, artifact_file="correlation_plot.png")
    #mlflow.log_figure(figure=information_gain_plot, artifact_file="information_gain_plot.png")

    # Guardamos el artefacto del modelo
    #artifact_path = "model"

    signature = infer_signature(X_train, model.predict(X_train))

    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path=artifact_path,
        signature=signature,
        serialization_format='cloudpickle',
        registered_model_name="mnist_784_model_dev",
        metadata={"model_data_version": 1}
    )

    # Obtenemos la ubicación del modelo guardado en MLFlow
    model_uri = mlflow.get_artifact_uri(artifact_path)

print(model_uri)


Registered model 'mnist_784_model_dev' already exists. Creating a new version of this model...
2024/04/27 20:22:24 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: mnist_784_model_dev, version 2


s3://mlflow/1/6059cf703acb4d48be9d61c849538926/artifacts/model


Created version '2' of model 'mnist_784_model_dev'.


## Testeando el modelo

Una vez que el modelo fue entrenado, podemos levantarlo y testearlo de una forma agnóstica a donde está guardado.

In [9]:
#loaded = mlflow.sklearn.load_model(model_uri)
loaded = mlflow.sklearn.load_model("mnist_784_model_dev")

OSError: No such file or directory: 'mnist_784_model_dev'

In [ ]:
import numpy as np
test_data = [-1.0431146438366603, 0.6689936080056726, 0.5955141571109206, 1.4218278518511829, -0.3955054753168235,
             -0.195684619877533, 1.445528359737701, -0.8782783888787548, -0.4354941703556927, -0.6313862911472252,
             1.0752906583803283, -0.0987729596649589, 0.957427107756338, 1.1071614388213236, -0.2991215208080594,
             -0.5494422557947561, -0.362142984170074, 4.690415759823429, -0.2253029545296664, 1.1980376111153852]
loaded.predict(np.array(test_data).reshape([1, -1]))

/Users/facundolucianna/Investigacion/env/notebook_env_python3_8/lib/python3.8/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but SVC was fitted with feature names
  warnings.warn(


array([1])

## Registramos el modelo 

Realizamos el registro del modelo en MLflow. En este registro se pone el modelo productivo que luego se usará para servir en formato on-line.

In [10]:
from mlflow import MlflowClient

client = MlflowClient()
name = "mnist_784_model_prod"
desc = "mnist_784"

# Creamos el modelo productivo
client.create_registered_model(name=name, description=desc)

# Guardamos como tag los hiper-parametros en la version del modelo
tags = model.get_params()
tags["model"] = type(model).__name__
tags["f1-score"] = f1_score

# Guardamos la version del modelo
result = client.create_model_version(
    name=name,
    source=model_uri,
    run_id=model_uri.split("/")[-3],
    tags=tags
)

# Y creamos como la version con el alias de champion para poder levantarlo en nuestro
# proceso de servicio del modelo on-line.
client.set_registered_model_alias(name, "champion", result.version)

2024/04/27 20:28:28 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: mnist_784_model_prod, version 1
